In [1]:
import os
import shutil
import asyncio
import tempfile
import threading
import contextlib

import unittest.mock as mock

import synapse.glob as s_glob
import synapse.common as s_common
import synapse.cortex as s_cortex
# import synapse.lib.cli as s_cli
import synapse.lib.cmdr as s_cmdr

import synapse.tests.utils as s_t_utils
print(f'Current pid: {os.getpid()}')
current_thread = threading.current_thread()
print(f'Current thread / thread.ident: {current_thread}')
print(type(current_thread))
loop = s_glob.initloop()
print(f'Current loop: {loop} / {id(loop)}')
glob_thrd = s_glob._glob_thrd
print(f'loop thread: {glob_thrd}')


Current pid: 7598
Current thread / thread.ident: <_MainThread(MainThread, started 140437956636800)>
<class 'threading._MainThread'>
getting running loop
_glob_loop is <_UnixSelectorEventLoop running=True closed=False debug=False>/140437776183704
hey we got _glob_thread <_MainThread(MainThread, started 140437956636800)>
Current loop: <_UnixSelectorEventLoop running=True closed=False debug=False> / 140437776183704
loop thread: <_MainThread(MainThread, started 140437956636800)>


In [ ]:
@contextlib.asynccontextmanager
async def getTempCoreProxy():
    with s_common.getTempDir() as dirn:
        print(f'Got tempdir: {dirn}')
        print('getting cortex....')
        async with await s_cortex.Cortex.anit(dirn) as core:
            print(f'Got cortex: {core}')
            print('Getting local proxy')
            core.conf['storm:log'] = True
            async with core.getLocalProxy() as prox:
                print(f'Got proxy: {prox}')
                yield prox
                print('closing down proxy/local dmon')
            print('closing down cortex')
        print('tearing down tempdir')
    print('done with getTempCortex')

acm = getTempCoreProxy()
proxy = await acm.__aenter__()
print(proxy, proxy.isfini)

print('preloading some data...')
async for mesg in await proxy.storm('[inet:ipv4=1.2.3.4 inet:ipv4=8.8.8.8]'):
    print(mesg)
    
cmdr = await s_cmdr.getItemCmdr(proxy)
print(f'Got cmdr: {cmdr}')

# await cmdr.runCmdLine('storm inet:ipv4')
# cmdr.runCmdLoop()
await cmdr.asyncRunCmdLoop()

await acm.__aexit__(None, None, None)
print(proxy, proxy.isfini)
    

Got tempdir: /tmp/tmpkl9ksvwn
getting cortex....


Executing storm query {[inet:ipv4=1.2.3.4 inet:ipv4=8.8.8.8]} as [None]


Got cortex: <synapse.cortex.Cortex object at 0x7fba1cc5fdd8>
Getting local proxy
listing on local loopback: tcp://127.0.0.1:0
Calling listen for tcp://127.0.0.1:0
Got proxy: <synapse.telepath.Proxy object at 0x7fba1e0f8b00>
<synapse.telepath.Proxy object at 0x7fba1e0f8b00> False
preloading some data...
('init', {'tick': 1544213041849, 'text': '[inet:ipv4=1.2.3.4 inet:ipv4=8.8.8.8]', 'task': 'd41a1efa5d3f9ab786b8f05882634818'})
('node:add', {'ndef': ('inet:ipv4', 16909060), 'user': '?', 'time': 1544213041852})
('prop:set', {'ndef': ('inet:ipv4', 16909060), 'prop': 'type', 'valu': 'unicast', 'oldv': None, 'user': '?', 'time': 1544213041852})
('prop:set', {'ndef': ('inet:ipv4', 16909060), 'prop': 'asn', 'valu': 0, 'oldv': None, 'user': '?', 'time': 1544213041852})
('node:add', {'ndef': ('inet:asn', 0), 'user': '?', 'time': 1544213041852})
('prop:set', {'ndef': ('inet:asn', 0), 'prop': 'name', 'valu': '??', 'oldv': None, 'user': '?', 'time': 1544213041852})
('prop:set', {'ndef': ('inet:asn

Executing storm query {inet:ipv4} as [None]


executing line
got coro: <coroutine object Cli.runCmdLine at 0x7fba1cb2f648>
waiting for result

cli - Running query inet:ipv4
got mesg: ('init', {'tick': 1544213047813, 'text': 'inet:ipv4', 'task': '4552dbcb596bab82dff05084fdef5522'})
got mesg: ('node', (('inet:ipv4', 16909060), {'iden': '20153b758f9d5eaaa38e4f4a65c36da797c3e59e549620fa7c4895e1a920991f', 'tags': {}, 'props': {'.created': 1544213041853, 'asn': 0, 'loc': '??', 'type': 'unicast'}, 'repr': '1.2.3.4', 'reprs': {'.created': '2018/12/07 20:04:01.853', 'asn': '0'}, 'path': {}}))
inet:ipv4=1.2.3.4
        .created = 2018/12/07 20:04:01.853
        :asn = 0
        :loc = ??
        :type = unicast
got mesg: ('node', (('inet:ipv4', 134744072), {'iden': 'ee6b92c9fd848a2cb00f3a3618148c512b58456b8b51fbed79251811597eeea3', 'tags': {}, 'props': {'.created': 1544213041853, 'asn': 0, 'loc': '??', 'type': 'unicast'}, 'repr': '8.8.8.8', 'reprs': {'.created': '2018/12/07 20:04:01.853', 'asn': '0'}, 'path': {}}))
inet:ipv4=8.8.8.8
       